# Weekly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
from datetime import datetime, timedelta
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:

%sql $Config.NAMEX_DATABASE_URI

Weekly running time.

In [ ]:
%%sql stat_weekly_submitted  <<
SELECT count(r.*) AS "Number of Names Submitted:" FROM requests r
WHERE date(r.submitted_date at time zone 'utc' at time zone 'pst')  > date(current_date - 1 - interval '1 weeks')
    AND date(r.submitted_date at time zone 'utc' at time zone 'pst')  <= date(current_date - 1)

In [ ]:
filename = 'weekly_totals_till_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'

with open(filename, 'w') as f:
    stat_weekly_submitted.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:

%%sql stat_weekly_completed  <<
WITH Detail AS 
(
    SELECT e.user_id     
     , (SELECT username FROM users u WHERE u.id=e.user_id) AS EXAMINER
     , count(e.*) FILTER (WHERE e.state_cd = 'APPROVED')  AS APPROVED
     , count(e.*) FILTER (WHERE e.state_cd = 'REJECTED')  AS REJECTED
     , count(e.*) FILTER (WHERE e.state_cd = 'CONDITIONAL')  AS CONDITIONAL
     , count(e.*) FILTER (WHERE e.state_cd = 'CANCELLED')  AS CANCELLED
     , count(r.*) FILTER (WHERE r.priority_cd = 'Y')  AS PRIORITIES
     , count(e.*) + count(r.*) FILTER (WHERE r.priority_cd = 'Y')   AS TOTAL                  
    FROM events e,
    requests r
    WHERE e.user_id != 1
    AND r.id = e.nr_id
    AND date(e.event_dt at time zone 'utc' at time zone 'pst')  > date(current_date - 1 - interval '1 weeks')
    AND date(e.event_dt at time zone 'utc' at time zone 'pst')  <= date(current_date - 1)
    AND e.state_cd in ('APPROVED','REJECTED','CONDITIONAL','CANCELLED')
    GROUP BY e.user_id
)

SELECT * from Detail WHERE examiner LIKE '%idir%'
UNION all
SELECT null 
  , 'SUM' EXAMINER
  , sum(APPROVED) AS APPROVED
  , sum(REJECTED) AS REJECTED
  , sum(CONDITIONAL) AS CONDITIONAL
  , sum(CANCELLED) AS CANCELLED
  , sum(PRIORITIES) AS PRIORITIES
  , sum(TOTAL) AS TOTAL
FROM Detail WHERE examiner LIKE '%idir%';

In [ ]:
edt = stat_weekly_completed.DataFrame()
edt['examiner'] = edt['examiner'].str.replace('idir/','')

In [ ]:
edt['approved+conditional_%'] = round(((edt.approved + edt.conditional) / edt.total).astype('float') * float(100), 2)
edt['rejected_%'] = round((edt.rejected / edt.total).astype('float') * float(100), 2)

Save to CSV

In [ ]:
with open(filename, 'a') as f:      
    f.write('\n\n Number of Names Processed:\n')
    edt.to_csv(f, sep=',', encoding='utf-8', index=False)    